In [ ]:
from control import lqr
from common_function import newmark, eigenvalue, newmark_one
import numpy as np
from matplotlib import pyplot as plt
import os, re
import pandas as pd
from matplotlib import font_manager  #matplotlib中 中文设置模块

In [ ]:
## 输出高清图像
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import platform
# 图像显示中文的问题，需要判断系统是windows还是苹果的
import matplotlib
import platform

sys_platform = platform.platform().lower()
if "windows" in sys_platform:
    font = {
        "family": "Times New Roman"
    }
    matplotlib.rc("font", **font)
else:
    font = {
        "family": "Arial Unicode MS"
    }
    matplotlib.rc("font", **font)
rc = {"mathtext.fontset": "stix", }

plt.rcParams.update(rc)

In [ ]:
wave = 0
dt = 0
name = "white_noise_190117_3.txt"
root = "./waves"
print(name)
wave_name = name.split('.')[0]
if name.endswith(".AT2"):
    filename = os.path.join(root, name)
    with open(filename, 'r') as fo:
        for i in range(3):
            fo.readline()
        line = fo.readline()
        target = re.search(r'DT= *(.*) *SEC', line, re.I | re.M)
        dt = float(target.group(1))
        wave = pd.read_table(filename, skiprows=4, sep="  ", header=None, engine='python')
        wave = wave.fillna(0)
        wave = wave.values
        size = wave.size
        wave = wave.reshape((size,))
if name.endswith(".txt"):
    filename = os.path.join(root, name)
    wave = np.loadtxt(filename)
    dt = 0.02
wave = wave / max(abs(wave)) * 0.3 * 9.8
wave_len = len(wave)
time = np.arange(0, wave_len * dt, dt)

In [ ]:
# # 生成随机白噪声
# num = 16000
# wave = np.zeros(num)
# for i in range(13000):
#     wave[i] = np.random.randn()
# dt = 0.02
# wave = wave / max(abs(wave)) * 0.3 * 9.8
# wave_len = len(wave)
# time = np.arange(0, wave_len * dt, dt)
plt.figure(figsize=(10, 4))
plt.xlim([0, max(time)])
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'加速度 $(m/s^2)$', fontname='SimSun', fontsize=10)
pic_name = r"人工波加速度时程"
plt.plot(time, wave)
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# 加速度插值
dt_s = dt / 3
t_s = np.arange(0, max(time) + 10, dt_s)
wave_s = np.interp(t_s, time, wave)
wave_len_s = len(wave_s)
wave_len_s

In [ ]:
m = 35  # 单位：kg
k = 3000  # 单位：N/m
M = np.array([[m]])
K = np.array([[k]])
zeta = 0.02  # 结构固有阻尼比

w, T, phi, gamma, M_par = eigenvalue(M, K)
w0 = w[0]

In [ ]:
T[0]

In [ ]:
c = 2 * zeta * m * w0

In [ ]:
mu = 1
ksi = 0.001
kpa = 20
md0 = mu * m
cd0 = 2 * ksi * m * w0
kd0 = kpa * k

In [ ]:
# 半主动惯容基本参数
min_max = mu * m
min_min = 0
cd_max = 2 * ksi * m * w0
cd_min = 2 * ksi * m * w0
kd = kpa * k

In [ ]:
# 状态空间矩阵
A = np.array([[0, 1], [-w0 ** 2, -2 * zeta * w0]], dtype=float)
B = np.array([[0, 0], [0, -1 / m]], dtype=float)

In [ ]:
# 原结构
M0 = np.array(m, ndmin=2, dtype=float)
C0 = np.array(c, ndmin=2, dtype=float)
K0 = np.array(k, ndmin=2, dtype=float)
E0 = np.array(1, ndmin=2, dtype=float)
y0, dy0, ddy0, ddy0_ab = newmark(K0, M0, C0, wave_s, dt_s, E0)

In [ ]:
# 被动控制结构
M = np.array([[m, 0], [0, md0]], ndmin=2, dtype=float)
K = np.array([[k + kd0, -kd0], [-kd0, kd0]], ndmin=2, dtype=float)
C = np.array([[c, 0], [0, cd0]], ndmin=2, dtype=float)
E = np.array([[1], [0]], ndmin=2, dtype=float)
u1, du1, _, ddu1_ab = newmark(K, M, C, wave_s, dt_s, E)

In [ ]:
plt.plot(t_s, ddy0_ab.reshape(-1))
plt.plot(t_s, ddu1_ab[0, :])

In [ ]:
plt.plot(t_s, dy0.reshape(-1))
plt.plot(t_s, du1[0, :])

In [ ]:
plt.plot(t_s, y0.reshape(-1))
plt.plot(t_s, u1[0, :])

In [ ]:
# 反馈增益矩阵
Q = np.diag([1, 0])
R = np.diag([1e-6, 1e-6])
D, _, _ = lqr(A, B, Q, R)
D = -D

In [ ]:
# 半主动控制结构
u2 = np.zeros([2, wave_len_s])
du2 = np.zeros([2, wave_len_s])
du2[1, 0] = -wave_s[0]
F = np.zeros([2, wave_len_s])
ft = np.zeros(wave_len_s)
ud = np.zeros(wave_len_s)
dud = np.zeros(wave_len_s)
ddud = np.zeros(wave_len_s)
fmax = np.zeros(wave_len_s)
fmin = np.zeros(wave_len_s)
fa = np.zeros(wave_len_s)
fmd = np.zeros(wave_len_s)

y2 = np.zeros(wave_len_s)
dy2 = np.zeros(wave_len_s)
ddy2 = np.zeros(wave_len_s)
ddy2_ab = np.zeros(wave_len_s)
ag_pre = 0

for i in range(wave_len_s - 1):
    # p = np.array([[0], [-wave_s[i]]])  # 当前荷载向量
    F[:, i, None] = np.dot(D, u2[:, i, None])  # 当前反馈力
    ft[i] = F[1, i]  # 当前目标出力
    ud[i] = u2[0, i] - ft[i] / kd  # 当前惯容位移
    dud[i] = (ud[i] - ud[i - 1]) / dt_s  # 当前惯容速度
    ddud[i] = (dud[i] - dud[i - 1]) / dt_s  # 当前惯容加速度

    # 当前出力上限
    fmax[i] = max(
        [min_max * ddud[i] + cd0 * dud[i],
         min_min * ddud[i] + cd0 * dud[i]])
    # 当前出力下限
    fmin[i] = min([min_max * ddud[i] + cd0 * dud[i],
                   min_min * ddud[i] + cd0 * dud[i]])
    while True:
        if ft[i] < fmin[i]:
            ft[i] = fmin[i]
        elif ft[i] > fmax[i]:
            ft[i] = fmax[i]
        else:
            fa[i] = ft[i]
            break
        ud[i] = u2[0, i] - ft[i] / kd
        dud[i] = (ud[i] - ud[i - 1]) / dt_s  # 当前惯容速度
        ddud[i] = (dud[i] - dud[i - 1]) / dt_s  # 当前惯容加速度
        # 当前出力上限
        fmax[i] = max(
            [min_max * ddud[i] + cd0 * dud[i],
             min_min * ddud[i] + cd0 * dud[i]])
        # 当前出力下限
        fmin[i] = min([min_max * ddud[i] + cd0 * dud[i],
                       min_min * ddud[i] + cd0 * dud[i]])
    # Newmark_beta法
    fmd[i] = fa[i] - cd0 * dud[i]
    ag = wave_s[i] + fa[i] / m
    z_acc = ag - ag_pre
    z_y = (-m * z_acc + (4 / dt) * m * dy2[i] + 2 * m * ddy2[i] + 2 * c * dy2[i]) / (k + 2 * c / dt + m * 4 / (dt ** 2))
    y2[i + 1] = y2[i] + z_y
    z_dy = 2 / dt * z_y - 2 * dy2[i]
    dy2[i + 1] = dy2[i] + z_dy
    z_ddy = 4 / (dt ** 2) * z_y - (4 / dt) * dy2[i] - 2 * ddy2[i]
    ddy2[i + 1] = ddy2[i] + z_ddy
    ddy2_ab[i] = ddy2[i] + ag

    ag_pre = ag
    u2[0, i + 1] = y2[i + 1]
    u2[1, i + 1] = dy2[i + 1]
    du2[0, i + 1] = dy2[i + 1]
    du2[1, i] = ddy2_ab[i]

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, fmax, label="出力上限")
plt.plot(t_s, fmin, label="出力下限")
plt.plot(t_s, fa, label="实际控制力")
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'控制力 $(kN)$', fontname='SimSun', fontsize=10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"控制力限值"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# plt.figure(figsize=(4,4))
# plt.xticks(fontsize=10)
# plt.yticks(fontsize=10)
# plt.plot(u1[1, :], du1[1, :] * cd0, label="被动控制系统")
# plt.plot(ud, dud * cd0, label="可变惯容半主动控制系统")
# plt.xlabel(r'位移 $(m)$',fontname = 'SimSun',fontsize = 10)
# plt.ylabel(r'阻尼力 $(kN)$',fontname = 'SimSun',fontsize = 10)
# # legend中文显示问题
# # font = font_manager.FontProperties(fname = 'C:/Windows/fonts/simsun.ttc')
# font = {
#     "family": "SimSun",
#     'weight':'normal',
#     "size": 8
#     }
# plt.legend(prop = font, loc='best')
# pic_name = r"滞回曲线对比"
# plt.savefig(r'./pic/{}.png'.format(pic_name),dpi=600, bbox_inches='tight')
# plt.show()
# plt.close()

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, y0.reshape(-1), label="原结构")
plt.plot(t_s, u1[0, :], label="被动控制系统")
plt.plot(t_s, y2, label="可变惯容半主动控制系统")
plt.xlim([0, max(t_s)])
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'位移 $(m)$', fontname='SimSun', fontsize=10)
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"位移对比"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()
print("被动控制位移减震比：{}".format(max(abs(u1[0, :])) / max(abs(y0.reshape(-1)))))
print("半主动控制位移减震比：{}".format(max(abs(y2)) / max(abs(y0.reshape(-1)))))

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, dy0.reshape(-1), label="原结构")
plt.plot(t_s, du1[0, :], label="被动控制系统")
plt.plot(t_s, dy2, label="可变惯容半主动控制系统")
plt.xlim([0, max(t_s)])
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'速度 $(m/s)$', fontname='SimSun', fontsize=10)
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"速度对比"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()
print("被动控制速度减震比：{}".format(max(abs(du1[0, :])) / max(abs(dy0.reshape(-1)))))
print("半主动控制速度减震比：{}".format(max(abs(dy2)) / max(abs(dy0.reshape(-1)))))

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, ddu1_ab[0, :], label="被动控制系统")
plt.plot(t_s, ddy0_ab.reshape(-1), label="原结构")
plt.plot(t_s, ddy2_ab, label="可变惯容半主动控制系统")
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'加速度 $(m/s^2)$', fontname='SimSun', fontsize=10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"加速度对比"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()
print("被动控制加速度减震比：{}".format(max(abs(ddu1_ab[0, :])) / max(abs(ddy0_ab.reshape(-1)))))
print("半主动控制加速度减震比：{}".format(max(abs(ddy2_ab)) / max(abs(ddy0_ab.reshape(-1)))))

In [ ]:
plt.plot(t_s, ft)

In [ ]:
plt.plot(t_s, ddud)

In [ ]:
y = u2[0, :]
dy = u2[1, :]
ddy_ab = du2[1, :]

In [ ]:
length = len(y)
columns = ["str_cur_disp", "str_cur_vel", "str_cur_acc", "str_pre_disp", "str_pre_vel", "str_pre_acc", "ud_pre",
           "ud_cur", "dud_pre", "dud_cur", "ddud_pre", "ddud_cur", "fmd_pre", "fmd_cur", "fa_pre", "fa_cur", "fa_next"]
numpy_data = np.zeros([length, len(columns)])
data = pd.DataFrame(data=numpy_data, columns=columns)

In [ ]:
# 构造样本数据
for i in range(1, length - 1):
    row_data = np.array(
        [y[i], dy[i], ddy_ab[i], y[i - 1], dy[i - 1], ddy_ab[i - 1], ud[i - 1], ud[i], dud[i - 1], dud[i], ddud[i - 1],
         ddud[i], fmd[i - 1], fmd[i], fa[i - 1], fa[i], fa[i + 1]])
    data.loc[i - 1] = row_data

In [ ]:
x_label = ["str_cur_disp", "str_cur_vel", "str_cur_acc", "str_pre_disp", "str_pre_vel",
           "str_pre_acc", "ud_pre",
           "ud_cur", "dud_pre", "dud_cur", "ddud_pre", "ddud_cur", "fmd_pre", "fmd_cur", "fa_pre", "fa_cur"]
y_label = "fa_next"
X = data.loc[:, x_label].values
y = data.loc[:, y_label].values

In [ ]:
data_len = data.shape[0]
train_len = int(0.6 * data_len)
X_train = X[0: train_len, :]
X_test = X[train_len: data_len, :]
y_train = y[0: train_len]
y_test = y[train_len: data_len]

In [ ]:
# 特征标准化，采用最大最小值标准化，转化后的值范围（0,1）
from sklearn.preprocessing import MinMaxScaler

min_max_scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
new_X_train = X_train
new_X_test = X_test
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler

# normalizer = Normalizer(copy=True, norm='l2').fit(new_X_train)
# new_X_train = normalizer.transform(new_X_train)
# new_X_test = normalizer.transform(new_X_test)
ss = StandardScaler().fit(new_X_train)
new_X_train = ss.transform(new_X_train)
new_X_test = ss.transform(new_X_test)

In [ ]:
# 模型选择及超参优化
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

# 五折交叉验证
num_folds = 5
random_state = 42
kf = KFold(n_splits=num_folds, random_state=42, shuffle=True)
GBoost = GradientBoostingRegressor(random_state=42)
score = -cross_val_score(GBoost, new_X_train, y_train, cv=kf, scoring="neg_mean_squared_error", n_jobs=-1).mean()
print(score)

In [ ]:
np.linspace(500, 3000, 6, dtype=int)

In [ ]:
# # 网格搜索
# from sklearn.model_selection import GridSearchCV
#
# param_grid = {'learning_rate': np.logspace(-3, 0, 4),
#               'subsample': np.array([0.5, 0.6, 0.7, 0.8, 0.9, 1], dtype=float),
#               'max_depth': np.array([3, 5, 8, 15, 25, 30], dtype=int),
#               'n_estimators': np.array([120, 300, 500, 800, 1200, 2000, 3000], dtype=int),
#               'criterion': ['friedman_mse', 'mse', 'mae'],
#               'min_samples_leaf': np.array([2, 5, 10, 15], dtype=int),
#               'min_samples_split': np.array([2, 5, 10, 15, 100], dtype=int),
#               'loss': ['ls', 'lad', 'huber', 'quantile'],
#               'random_state': [random_state]}
# gs = GridSearchCV(GBoost, param_grid, scoring='neg_mean_squared_error', n_jobs=-1, cv=kf, verbose=False)
# gs.fit(new_X_train, y_train)
# gs_test_score = mean_squared_error(y_test, gs.predict(new_X_test))

In [ ]:
# # 随机搜索
# from sklearn.model_selection import RandomizedSearchCV
# from scipy.stats import randint
#
# param_grid_rand = {'learning_rate': np.logspace(-5, 0, 100),
#                    'subsample': np.array([0.5, 0.6, 0.7, 0.8, 0.9, 1], dtype=float),
#                    'max_depth': randint(2, 20),
#                    'n_estimators': randint(100, 2000),
#                    'criterion': ['friedman_mse', 'mae'],
#                    'min_samples_leaf': randint(2, 20),
#                    'min_samples_split': randint(2, 20),
#                    'loss': ['ls', 'lad', 'huber', 'quantile'],
#                    'random_state': [random_state]}
#
# rs = RandomizedSearchCV(GBoost, param_grid_rand, n_iter=50, scoring='neg_mean_squared_error', n_jobs=-1, cv=kf,
#                         verbose=False, random_state=random_state)
#
# rs.fit(new_X_train, y_train)
# rs_test_score = mean_squared_error(y_test, rs.predict(new_X_test))

In [ ]:
# 贝叶斯优化
from skopt import BayesSearchCV
param_grid = {'learning_rate': np.logspace(-3, 0, 4),
              'subsample': np.array([0.5, 0.6, 0.7, 0.8, 0.9, 1], dtype=float),
              'max_depth': np.array([3, 5, 8, 15, 25, 30], dtype=int),
              'n_estimators': np.array([120, 300, 500, 800, 1200, 2000, 3000], dtype=int),
              'criterion': ['friedman_mse', 'mse'],
              'min_samples_leaf': np.array([2, 5, 10, 15], dtype=int),
              'min_samples_split': np.array([2, 5, 10, 15, 100], dtype=int),
              'loss': ['ls', 'lad', 'huber', 'quantile'],
              'random_state': [random_state]}
opt = BayesSearchCV(GBoost, param_grid, n_iter=5, n_jobs=30, cv=3)
opt.fit(new_X_train, y_train)

In [ ]:
# opt = GBoost
model = opt
# opt.fit(new_X_train, y_train)
pred = opt.predict(new_X_test)

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)
GBoost.fit(new_X_train, y_train)


train_pred = GBoost.predict(new_X_train)
pred = GBoost.predict(new_X_test)
print(rmsle(y_train, train_pred))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


def plot_regression_results(ax, y_true, y_pred, title, scores):
    """预测目标与真实目标的散点图。"""
    ax.plot([y_true.min(), y_true.max()],
            [y_true.min(), y_true.max()], '--r', linewidth=2)
    ax.scatter(y_true, y_pred, alpha=0.2)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    ax.set_xlim([y_true.min(), y_true.max()])
    ax.set_ylim([y_true.min(), y_true.max()])
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')
    extra = plt.Rectangle((0, 0), 0, 0, fc="w", fill=False,
                          edgecolor='none', linewidth=0)
    ax.legend([extra], [scores], loc='upper left')
    ax.set_title(title)


def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_squared_error

r2 = r2_score(y_test, pred)
mse = mean_squared_error(y_test, pred)
mape = mean_absolute_percentage_error(y_test, pred)

plt.figure(figsize=(10, 8))
ax = plt.gca()
plot_regression_results(
    ax, y_test, pred,
    "Stacking Regressor",
    (r'$R^2={:.2f}$' + '\n' + r'$MAPE={:.2f}$' + '\n' + r'$MSE={:.2f}$')
    .format(r2, mse, mape))


In [ ]:
# 半主动控制结构
u2_ml = np.zeros([2, wave_len_s])
du2_ml = np.zeros([2, wave_len_s])
du2_ml[1, 0] = -wave_s[0]
F = np.zeros([2, wave_len_s])
ft_ml = np.zeros(wave_len_s)
ud_ml = np.zeros(wave_len_s)
dud_ml = np.zeros(wave_len_s)
ddud_ml = np.zeros(wave_len_s)
fmax_ml = np.zeros(wave_len_s)
fmin_ml = np.zeros(wave_len_s)
fa_ml = np.zeros(wave_len_s)
fmd_ml = np.zeros(wave_len_s)

str_pre_disp = 0
str_pre_vel = 0
str_pre_acc = 0
str_cur_disp = 0
str_cur_vel = 0
str_cur_acc = 0
ft_ml_pre = 0
ft_ml_next = 0
fa_ml_pre = 0
fa_ml_next = 0
fmd_ml_pre = 0
fmd_ml_cur = 0

ud_ml_pre = 0
ud_ml_cur = 0
dud_ml_pre = 0
dud_ml_cur = 0
ddud_ml_pre = 0
ddud_ml_cur = 0

y2_ml = np.zeros(wave_len_s)
dy2_ml = np.zeros(wave_len_s)
ddy2_ml = np.zeros(wave_len_s)
ddy2_ab_ml = np.zeros(wave_len_s)
ag_pre = 0
for i in range(wave_len_s - 1):
    str_cur_disp = y2_ml[i - 1]
    str_cur_vel = dy2_ml[i - 1]
    str_cur_acc = ddy2_ab_ml[i - 1]
    fa_ml_cur = fa_ml[i - 1]
    fmd_ml_cur = fmd_ml[i - 1]
    ud_ml_cur = ud_ml[i - 1]
    dud_ml_cur = dud_ml[i - 1]
    ddud_ml_cur = ddud_ml[i - 1]
    x = np.array([str_cur_disp, str_cur_vel, str_cur_acc, str_pre_disp, str_pre_vel, str_pre_acc, ud_ml_pre, ud_ml_cur,
                  dud_ml_pre, dud_ml_cur, ddud_ml_pre, ddud_ml_cur, fmd_ml_pre, fmd_ml_cur, fa_ml_pre, fa_ml_cur],
                 ndmin=2)
    x = ss.transform(x)
    ft_ml[i] = model.predict(x)[0]

    fa_ml_pre = fa_ml_cur
    str_pre_disp = str_cur_disp
    str_pre_vel = str_cur_vel
    str_pre_acc = str_cur_acc
    fmd_ml_pre = fmd_ml_cur
    ud_ml_pre = ud_ml_cur
    dud_ml_pre = dud_ml_cur
    ddud_ml_pre = ddud_ml_cur

    ud_ml[i] = u2_ml[0, i] - ft_ml[i] / kd  # 当前惯容位移
    dud_ml[i] = (ud_ml[i] - ud_ml[i - 1]) / dt_s  # 当前惯容速度
    ddud_ml[i] = (dud_ml[i] - dud_ml[i - 1]) / dt_s  # 当前惯容加速度

    # 当前出力上限
    fmax_ml[i] = max(
        [min_max * ddud_ml[i] + cd_max * dud_ml[i],
         min_min * ddud_ml[i] + cd_min * dud_ml[i]])
    # 当前出力下限
    fmin_ml[i] = min([min_max * ddud_ml[i] + cd_max * dud_ml[i],
                      min_min * ddud_ml[i] + cd_min * dud_ml[i]])
    while True:
        if ft_ml[i] < fmin_ml[i]:
            ft_ml[i] = fmin_ml[i]
        elif ft_ml[i] > fmax_ml[i]:
            ft_ml[i] = fmax_ml[i]
        else:
            fa_ml[i] = ft_ml[i]
            break
        ud_ml[i] = u2_ml[0, i] - ft_ml[i] / kd  # 当前惯容位移
        dud_ml[i] = (ud_ml[i] - ud_ml[i - 1]) / dt_s  # 当前惯容速度
        ddud_ml[i] = (dud_ml[i] - dud_ml[i - 1]) / dt_s  # 当前惯容加速度

        # 当前出力上限
        fmax_ml[i] = max(
            [min_max * ddud_ml[i] + cd_max * dud_ml[i],
             min_min * ddud_ml[i] + cd_min * dud_ml[i]])
        # 当前出力下限
        fmin_ml[i] = min([min_max * ddud_ml[i] + cd_max * dud_ml[i],
                          min_min * ddud_ml[i] + cd_min * dud_ml[i]])

    # ud_ml_pre = ud_ml[i]
    # dud_ml_pre = dud_ml[i]
    fmd_ml[i] = fa_ml[i] - cd0 * dud_ml[i]
    # Newmark_beta法
    ag = wave_s[i] + fa_ml[i] / m
    z_acc = ag - ag_pre
    z_y = (-m * z_acc + (4 / dt) * m * dy2_ml[i] + 2 * m * ddy2_ml[i] + 2 * c * dy2_ml[i]) / (
            k + 2 * c / dt + m * 4 / (dt ** 2))
    y2_ml[i + 1] = y2_ml[i] + z_y
    z_dy = 2 / dt * z_y - 2 * dy2_ml[i]
    dy2_ml[i + 1] = dy2_ml[i] + z_dy
    z_ddy = 4 / (dt ** 2) * z_y - (4 / dt) * dy2_ml[i] - 2 * ddy2_ml[i]
    ddy2_ml[i + 1] = ddy2_ml[i] + z_ddy
    ddy2_ab_ml[i] = ddy2_ml[i] + ag

    ag_pre = ag
    u2_ml[0, i + 1] = y2_ml[i + 1]
    u2_ml[1, i + 1] = dy2_ml[i + 1]
    du2_ml[0, i] = dy2_ml[i]
    du2_ml[1, i] = ddy2_ab_ml[i]

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, y0.reshape(-1), label="原结构")
plt.plot(t_s, u1[0, :], label="被动控制系统")
plt.plot(t_s, y2, label="可变惯容半主动控制系统")
plt.plot(t_s, y2_ml, label="基于机器学习的可变惯容半主动控制系统")
plt.xlim([0, max(t_s)])
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'位移 $(m)$', fontname='SimSun', fontsize=10)
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"位移对比_ml"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()
print("被动控制位移减震比：{}".format(max(abs(u1[0, :])) / max(abs(y0.reshape(-1)))))
print("半主动控制位移减震比：{}".format(max(abs(y2)) / max(abs(y0.reshape(-1)))))
print("机器学习半主动控制位移减震比：{}".format(max(abs(y2_ml)) / max(abs(y0.reshape(-1)))))

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, dy0.reshape(-1), label="原结构")
plt.plot(t_s, du1[0, :], label="被动控制系统")
plt.plot(t_s, dy2, label="可变惯容半主动控制系统")
plt.plot(t_s, dy2_ml, label="基于机器学习的可变惯容半主动控制系统")
plt.xlim([0, max(t_s)])
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'速度 $(m/s)$', fontname='SimSun', fontsize=10)
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"速度对比_ml"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()
print("被动控制速度减震比：{}".format(max(abs(du1[0, :])) / max(abs(dy0.reshape(-1)))))
print("半主动控制速度减震比：{}".format(max(abs(dy2)) / max(abs(dy0.reshape(-1)))))
print("机器学习半主动速度位移减震比：{}".format(max(abs(dy2_ml)) / max(abs(dy0.reshape(-1)))))

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, ddy0_ab.reshape(-1), label="原结构")
plt.plot(t_s, ddu1_ab[0, :], label="被动控制系统")
plt.plot(t_s, ddy2_ab, label="可变惯容半主动控制系统")
plt.plot(t_s, ddy2_ab_ml, label="基于机器学习的可变惯容半主动控制系统")
plt.xlim([0, max(t_s)])
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'加速度 $(m/s^2)$', fontname='SimSun', fontsize=10)
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"加速度对比_ml"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()
print("被动控制加速度减震比：{}".format(max(abs(ddu1_ab[0, :])) / max(abs(ddy0_ab.reshape(-1)))))
print("半主动控制加速度减震比：{}".format(max(abs(ddy2_ab)) / max(abs(ddy0_ab.reshape(-1)))))
print("机器学习半主动加速度位移减震比：{}".format(max(abs(ddy2_ab_ml)) / max(abs(ddy0_ab.reshape(-1)))))

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, fmax_ml)
plt.plot(t_s, fmin_ml)
plt.plot(t_s, fa_ml)
plt.xlim([0, max(t_s)])

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, fa, label="可变惯容半主动控制系统")
plt.plot(t_s, fa_ml, label="基于机器学习的可变惯容半主动控制系统")
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'控制力 $(kN)$', fontname='SimSun', fontsize=10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"控制力对比_ml"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, ud, label="可变惯容半主动控制系统")
plt.plot(t_s, ud_ml, label="基于机器学习的可变惯容半主动控制系统")

plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'位移 $(m)$', fontname='SimSun', fontsize=10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"惯容元件位移对比_ml"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, dud, label="可变惯容半主动控制系统")
plt.plot(t_s, dud_ml, label="基于机器学习的可变惯容半主动控制系统")
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'速度 $(m/s)$', fontname='SimSun', fontsize=10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"惯容元件速度对比_ml"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, ddud, label="可变惯容半主动控制系统")
plt.plot(t_s, ddud_ml, label="基于机器学习的可变惯容半主动控制系统")
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'加速度 $(m/s)$', fontname='SimSun', fontsize=10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"惯容元件加速度对比_ml"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# 用另外一条波
wave = 0
dt = 0
name = "RSN55_SFERN_BVP090.AT2"
root = "./waves"
print(name)
wave_name = name.split('.')[0]
if name.endswith(".AT2"):
    filename = os.path.join(root, name)
    with open(filename, 'r') as fo:
        for i in range(3):
            fo.readline()
        line = fo.readline()
        target = re.search(r'DT= *(.*) *SEC', line, re.I | re.M)
        dt = float(target.group(1))
        wave = pd.read_table(filename, skiprows=4, sep="  ", header=None, engine='python')
        wave = wave.fillna(0)
        wave = wave.values
        size = wave.size
        wave = wave.reshape((size,))
if name.endswith(".txt"):
    filename = os.path.join(root, name)
    wave = np.loadtxt(filename)
    dt = 0.02
wave = wave / max(abs(wave)) * 0.3 * 9.8
wave_len = len(wave)
time = np.arange(0, wave_len * dt, dt)

In [ ]:
# 加速度插值
dt_s = dt * 1
t_s = np.arange(0, max(time) + 10, dt_s)
wave_s = np.interp(t_s, time, wave)
wave_len_s = len(wave_s)
wave_len_s

In [ ]:
M0 = np.array(m, ndmin=2, dtype=float)
C0 = np.array(c, ndmin=2, dtype=float)
K0 = np.array(k, ndmin=2, dtype=float)
E0 = np.array(1, ndmin=2, dtype=float)
y0, dy0, ddy0, ddy0_ab = newmark(K0, M0, C0, wave_s, dt_s, E0)

In [ ]:
# 被动控制结构
M = np.array([[m, 0], [0, md0]], ndmin=2, dtype=float)
K = np.array([[k + kd0, -kd0], [-kd0, kd0]], ndmin=2, dtype=float)
C = np.array([[c, 0], [0, cd0]], ndmin=2, dtype=float)
E = np.array([[1], [0]], ndmin=2, dtype=float)
u1, du1, _, ddu1_ab = newmark(K, M, C, wave_s, dt_s, E)

In [ ]:
plt.plot(t_s, y0.reshape(-1))
plt.plot(t_s, u1[0, :])

In [ ]:
# 半主动控制结构
u2 = np.zeros([2, wave_len_s])
du2 = np.zeros([2, wave_len_s])
du2[1, 0] = -wave_s[0]
F = np.zeros([2, wave_len_s])
ft = np.zeros(wave_len_s)
ud = np.zeros(wave_len_s)
dud = np.zeros(wave_len_s)
ddud = np.zeros(wave_len_s)
fmax = np.zeros(wave_len_s)
fmin = np.zeros(wave_len_s)
fa = np.zeros(wave_len_s)
ud_pre = 0
dud_pre = 0

y2 = np.zeros(wave_len_s)
dy2 = np.zeros(wave_len_s)
ddy2 = np.zeros(wave_len_s)
ddy2_ab = np.zeros(wave_len_s)
ag_pre = 0

for i in range(wave_len_s - 1):
    # p = np.array([[0], [-wave_s[i]]])  # 当前荷载向量
    F[:, i, None] = np.dot(D, u2[:, i, None])  # 当前反馈力
    ft[i] = F[1, i]  # 当前目标出力
    ud[i] = u2[0, i] - ft[i] / kd  # 当前惯容位移
    dud[i] = (ud[i] - ud_pre) / dt_s  # 当前惯容速度
    ddud[i] = (dud[i] - dud_pre) / dt_s  # 当前惯容加速度

    # 当前出力上限
    fmax[i] = max(
        [min_max * ddud[i] + cd_max * dud[i],
         min_max * ddud[i] + cd_min * dud[i],
         min_min * ddud[i] + cd_max * dud[i],
         min_min * ddud[i] + cd_min * dud[i]])
    # 当前出力下限
    fmin[i] = min([min_max * ddud[i] + cd_max * dud[i],
                   min_max * ddud[i] + cd_min * dud[i],
                   min_min * ddud[i] + cd_max * dud[i],
                   min_min * ddud[i] + cd_min * dud[i]])
    while True:
        if ft[i] < fmin[i]:
            ft[i] = fmin[i]
        elif ft[i] > fmax[i]:
            ft[i] = fmax[i]
        else:
            fa[i] = ft[i]
            break
        ud[i] = u2[0, i] - ft[i] / kd
        dud[i] = (ud[i] - ud_pre) / dt_s  # 当前惯容速度
        ddud[i] = (dud[i] - dud_pre) / dt_s  # 当前惯容加速度
        # 当前出力上限
        fmax[i] = max(
            [min_max * ddud[i] + cd_max * dud[i],
             min_max * ddud[i] + cd_min * dud[i],
             min_min * ddud[i] + cd_max * dud[i],
             min_min * ddud[i] + cd_min * dud[i]])
        # 当前出力下限
        fmin[i] = min([min_max * ddud[i] + cd_max * dud[i],
                       min_max * ddud[i] + cd_min * dud[i],
                       min_min * ddud[i] + cd_max * dud[i],
                       min_min * ddud[i] + cd_min * dud[i]])
    ud_pre = ud[i]
    dud_pre = dud[i]
    # Newmark_beta法
    ag = wave_s[i] + fa[i] / m
    z_acc = ag - ag_pre
    z_y = (-m * z_acc + (4 / dt) * m * dy2[i] + 2 * m * ddy2[i] + 2 * c * dy2[i]) / (k + 2 * c / dt + m * 4 / (dt ** 2))
    y2[i + 1] = y2[i] + z_y
    z_dy = 2 / dt * z_y - 2 * dy2[i]
    dy2[i + 1] = dy2[i] + z_dy
    z_ddy = 4 / (dt ** 2) * z_y - (4 / dt) * dy2[i] - 2 * ddy2[i]
    ddy2[i + 1] = ddy2[i] + z_ddy
    ddy2_ab[i] = ddy2[i] + ag

    ag_pre = ag
    u2[0, i + 1] = y2[i + 1]
    u2[1, i + 1] = dy2[i + 1]
    du2[0, i + 1] = dy2[i + 1]
    du2[1, i + 1] = ddy2_ab[i + 1]
    # du2[:, i + 1, None] = np.dot(A, u2[..., i, None]) + np.dot(B, F[..., i, None])  + p
    # u2[..., i + 1] = u2[..., i] + du2[..., i + 1] * dt_s


In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, y0.reshape(-1), label="原结构")
plt.plot(t_s, u1[0, :], label="被动控制系统")
plt.plot(t_s, y2, label="可变惯容半主动控制系统")
plt.xlim([0, max(t_s)])
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'位移 $(m)$', fontname='SimSun', fontsize=10)
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"位移对比_new"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()
print("被动控制位移减震比：{}".format(max(abs(u1[0, :])) / max(abs(y0.reshape(-1)))))
print("半主动控制位移减震比：{}".format(max(abs(y2)) / max(abs(y0.reshape(-1)))))

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, dy0.reshape(-1), label="原结构")
plt.plot(t_s, du1[0, :], label="被动控制系统")
plt.plot(t_s, dy2, label="可变惯容半主动控制系统")
plt.xlim([0, max(t_s)])
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'速度 $(m/s)$', fontname='SimSun', fontsize=10)
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"速度对比_new"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()
print("被动控制速度减震比：{}".format(max(abs(du1[0, :])) / max(abs(dy0.reshape(-1)))))
print("半主动控制速度减震比：{}".format(max(abs(dy2)) / max(abs(dy0.reshape(-1)))))

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, ddy0_ab.reshape(-1), label="原结构")
plt.plot(t_s, ddu1_ab[0, :], label="被动控制系统")
plt.plot(t_s, ddy2_ab, label="可变惯容半主动控制系统")
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'加速度 $(m/s^2)$', fontname='SimSun', fontsize=10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"加速度对比_new"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()
print("被动控制加速度减震比：{}".format(max(abs(ddu1_ab[0, :])) / max(abs(ddy0_ab.reshape(-1)))))
print("半主动控制加速度减震比：{}".format(max(abs(ddy2_ab)) / max(abs(ddy0_ab.reshape(-1)))))

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, fmax, label="出力上限")
plt.plot(t_s, fmin, label="出力下限")
plt.plot(t_s, fa, label="实际控制力")
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'控制力 $(kN)$', fontname='SimSun', fontsize=10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"控制力限值_new"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
# 半主动控制结构
u2_ml = np.zeros([2, wave_len_s])
du2_ml = np.zeros([2, wave_len_s])
du2_ml[1, 0] = -wave_s[0]
F = np.zeros([2, wave_len_s])
ft_ml = np.zeros(wave_len_s)
ud_ml = np.zeros(wave_len_s)
dud_ml = np.zeros(wave_len_s)
ddud_ml = np.zeros(wave_len_s)
fmax_ml = np.zeros(wave_len_s)
fmin_ml = np.zeros(wave_len_s)
fa_ml = np.zeros(wave_len_s)
fmd_ml = np.zeros(wave_len_s)

str_pre_disp = 0
str_pre_vel = 0
str_pre_acc = 0
str_cur_disp = 0
str_cur_vel = 0
str_cur_acc = 0
ft_ml_pre = 0
ft_ml_next = 0
fa_ml_pre = 0
fa_ml_next = 0
fmd_ml_pre = 0
fmd_ml_cur = 0

ud_ml_pre = 0
ud_ml_cur = 0
dud_ml_pre = 0
dud_ml_cur = 0
ddud_ml_pre = 0
ddud_ml_cur = 0

y2_ml = np.zeros(wave_len_s)
dy2_ml = np.zeros(wave_len_s)
ddy2_ml = np.zeros(wave_len_s)
ddy2_ab_ml = np.zeros(wave_len_s)
ag_pre = 0
for i in range(wave_len_s - 1):
    str_cur_disp = y2_ml[i - 1]
    str_cur_vel = dy2_ml[i - 1]
    str_cur_acc = ddy2_ab_ml[i - 1]
    fa_ml_cur = fa_ml[i - 1]
    fmd_ml_cur = fmd_ml[i - 1]
    ud_ml_cur = ud_ml[i - 1]
    dud_ml_cur = dud_ml[i - 1]
    ddud_ml_cur = ddud_ml[i - 1]
    x = np.array([str_cur_disp, str_cur_vel, str_cur_acc, str_pre_disp, str_pre_vel, str_pre_acc, ud_ml_pre, ud_ml_cur,
                  dud_ml_pre, dud_ml_cur, ddud_ml_pre, ddud_ml_cur, fmd_ml_pre, fmd_ml_cur, fa_ml_pre, fa_ml_cur],
                 ndmin=2)
    x = ss.transform(x)
    ft_ml[i] = opt.predict(x)[0]

    fa_ml_pre = fa_ml_cur
    str_pre_disp = str_cur_disp
    str_pre_vel = str_cur_vel
    str_pre_acc = str_cur_acc
    fmd_ml_pre = fmd_ml_cur
    ud_ml_pre = ud_ml_cur
    dud_ml_pre = dud_ml_cur
    ddud_ml_pre = ddud_ml_cur

    ud_ml[i] = u2_ml[0, i] - ft_ml[i] / kd  # 当前惯容位移
    dud_ml[i] = (ud_ml[i] - ud_ml[i - 1]) / dt_s  # 当前惯容速度
    ddud_ml[i] = (dud_ml[i] - dud_ml[i - 1]) / dt_s  # 当前惯容加速度

    # 当前出力上限
    fmax_ml[i] = max(
        [min_max * ddud_ml[i] + cd_max * dud_ml[i],
         min_min * ddud_ml[i] + cd_min * dud_ml[i]])
    # 当前出力下限
    fmin_ml[i] = min([min_max * ddud_ml[i] + cd_max * dud_ml[i],
                      min_min * ddud_ml[i] + cd_min * dud_ml[i]])
    while True:
        if ft_ml[i] < fmin_ml[i]:
            ft_ml[i] = fmin_ml[i]
        elif ft_ml[i] > fmax_ml[i]:
            ft_ml[i] = fmax_ml[i]
        else:
            fa_ml[i] = ft_ml[i]
            break
        ud_ml[i] = u2_ml[0, i] - ft_ml[i] / kd  # 当前惯容位移
        dud_ml[i] = (ud_ml[i] - ud_ml[i - 1]) / dt_s  # 当前惯容速度
        ddud_ml[i] = (dud_ml[i] - dud_ml[i - 1]) / dt_s  # 当前惯容加速度

        # 当前出力上限
        fmax_ml[i] = max(
            [min_max * ddud_ml[i] + cd_max * dud_ml[i],
             min_min * ddud_ml[i] + cd_min * dud_ml[i]])
        # 当前出力下限
        fmin_ml[i] = min([min_max * ddud_ml[i] + cd_max * dud_ml[i],
                          min_min * ddud_ml[i] + cd_min * dud_ml[i]])

    # ud_ml_pre = ud_ml[i]
    # dud_ml_pre = dud_ml[i]
    fmd_ml[i] = fa_ml[i] - cd0 * dud_ml[i]
    # Newmark_beta法
    ag = wave_s[i] + fa_ml[i] / m
    z_acc = ag - ag_pre
    z_y = (-m * z_acc + (4 / dt) * m * dy2_ml[i] + 2 * m * ddy2_ml[i] + 2 * c * dy2_ml[i]) / (
            k + 2 * c / dt + m * 4 / (dt ** 2))
    y2_ml[i + 1] = y2_ml[i] + z_y
    z_dy = 2 / dt * z_y - 2 * dy2_ml[i]
    dy2_ml[i + 1] = dy2_ml[i] + z_dy
    z_ddy = 4 / (dt ** 2) * z_y - (4 / dt) * dy2_ml[i] - 2 * ddy2_ml[i]
    ddy2_ml[i + 1] = ddy2_ml[i] + z_ddy
    ddy2_ab_ml[i] = ddy2_ml[i] + ag

    ag_pre = ag
    u2_ml[0, i + 1] = y2_ml[i + 1]
    u2_ml[1, i + 1] = dy2_ml[i + 1]
    du2_ml[0, i] = dy2_ml[i]
    du2_ml[1, i] = ddy2_ab_ml[i]

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, y0.reshape(-1), label="原结构")
plt.plot(t_s, u1[0, :], label="被动控制系统")
plt.plot(t_s, y2, label="可变惯容半主动控制系统")
plt.plot(t_s, y2_ml, label="基于机器学习的可变惯容半主动控制系统")
plt.xlim([0, max(t_s)])
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'位移 $(m)$', fontname='SimSun', fontsize=10)
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"位移对比_ml_new"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()
print("被动控制位移减震比：{}".format(max(abs(u1[0, :])) / max(abs(y0.reshape(-1)))))
print("半主动控制位移减震比：{}".format(max(abs(y2)) / max(abs(y0.reshape(-1)))))
print("机器学习半主动控制位移减震比：{}".format(max(abs(y2_ml)) / max(abs(y0.reshape(-1)))))

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, dy0.reshape(-1), label="原结构")
plt.plot(t_s, du1[0, :], label="被动控制系统")
plt.plot(t_s, dy2, label="可变惯容半主动控制系统")
plt.plot(t_s, dy2_ml, label="基于机器学习的可变惯容半主动控制系统")
plt.xlim([0, max(t_s)])
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'速度 $(m/s)$', fontname='SimSun', fontsize=10)
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"速度对比_ml_new"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()
print("被动控制速度减震比：{}".format(max(abs(du1[0, :])) / max(abs(dy0.reshape(-1)))))
print("半主动控制速度减震比：{}".format(max(abs(dy2)) / max(abs(dy0.reshape(-1)))))
print("机器学习半主动速度位移减震比：{}".format(max(abs(dy2_ml)) / max(abs(dy0.reshape(-1)))))

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, ddy0_ab.reshape(-1), label="原结构")
plt.plot(t_s, ddu1_ab[0, :], label="被动控制系统")
plt.plot(t_s, ddy2_ab, label="可变惯容半主动控制系统")
plt.plot(t_s, ddy2_ab_ml, label="基于机器学习的可变惯容半主动控制系统")
plt.xlim([0, max(t_s)])
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'加速度 $(m/s^2)$', fontname='SimSun', fontsize=10)
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"加速度对比_ml_new"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()
print("被动控制加速度减震比：{}".format(max(abs(ddu1_ab[0, :])) / max(abs(ddy0_ab.reshape(-1)))))
print("半主动控制加速度减震比：{}".format(max(abs(ddy2_ab)) / max(abs(ddy0_ab.reshape(-1)))))
print("机器学习半主动加速度位移减震比：{}".format(max(abs(ddy2_ab_ml)) / max(abs(ddy0_ab.reshape(-1)))))

In [ ]:
plt.plot(t_s, fmax_ml)
plt.plot(t_s, fmin_ml)
plt.plot(t_s, fa_ml)

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, fa, label="可变惯容半主动控制系统")
plt.plot(t_s, fa_ml, label="基于机器学习的可变惯容半主动控制系统")
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'控制力 $(kN)$', fontname='SimSun', fontsize=10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"控制力对比_ml_new"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, ud, label="可变惯容半主动控制系统")
plt.plot(t_s, ud_ml, label="基于机器学习的可变惯容半主动控制系统")

plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'位移 $(m)$', fontname='SimSun', fontsize=10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"惯容元件位移对比_ml_new"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, dud, label="可变惯容半主动控制系统")
plt.plot(t_s, dud_ml, label="基于机器学习的可变惯容半主动控制系统")
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'速度 $(m/s)$', fontname='SimSun', fontsize=10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"惯容元件速度对比_ml_new"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()

In [ ]:
plt.figure(figsize=(10, 4))
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.plot(t_s, ddud, label="可变惯容半主动控制系统")
plt.plot(t_s, ddud_ml, label="基于机器学习的可变惯容半主动控制系统")
plt.xlabel(r'时间 $(s)$', fontname='SimSun', fontsize=10)
plt.ylabel(r'加速度 $(m/s)$', fontname='SimSun', fontsize=10)
plt.xlim([0, max(t_s)])
font = {
    "family": "SimSun",
    'weight': 'normal',
    "size": 8
}
plt.legend(prop=font, loc='best')
pic_name = r"惯容元件加速度对比_ml_new"
plt.savefig(r'./pic/{}.png'.format(pic_name), dpi=600, bbox_inches='tight')
plt.show()
plt.close()